In [118]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
import torchvision.transforms as transforms


In [119]:
device = torch.device('cuda' if torch.cuda.is_available() else  'cpu')

In [120]:
input_size = 28
num_class = 10
hidden_size = 256
num_layer = 2
sequence_length = 28
batch_size = 64
learning_rate =0.001
num_epochs = 5

In [ ]:
# in gru there is two gate a reset gate and update gate 
# reset gate = what memory should it forget
#update gate = what memory should it retain 
#input_size = feautures a one imestamp
#hidden_size = how much memory it have
class GRU(nn.Module):
    def __init__(self ,input_size , hiiden_size ,num_layer , num_class  ):
        super(GRU , self).__init__()
        self.hidden_size = hiiden_size
        self.num_layers = num_layer
        self.Gru = nn.GRU(input_size=input_size , hidden_size=hidden_size , num_layers=num_layer , batch_first=True , bidirectional=False)
        self.linear = nn.Linear(hiiden_size , num_class)
    def forward(self, x):
        out, h_n = self.Gru(x)  # out: [B, T, H], h_n: [num_layers, B, H]
        last_hidden = h_n[-1]   # take the last layer hidden state
        logits = self.linear(last_hidden)
        return logits 

In [122]:
test_data = MNIST(root='/dataset' , train=False , transform=transforms.ToTensor() , download=True)
train_data = MNIST(root='/dataset' ,train=True , transform=transforms.ToTensor() , download=True )
test_loader  = DataLoader(test_data , shuffle=True , batch_size=batch_size)
train_loader = DataLoader(train_data,shuffle=True , batch_size=batch_size )

In [123]:
model=GRU(input_size=input_size , hiiden_size=hidden_size ,num_layer=num_layer , num_class=num_class).to(device=device)

In [124]:
critision = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters() , lr=learning_rate)

In [125]:
for epochs in range(num_epochs):
    for batch_idx , (data , targets) in enumerate(train_loader):
        data = data.to(device = device).squeeze(1)
        targets = targets.to(device=device)
        scores = model(data)
        loss = critision(scores,targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [129]:
def check_accuracy(loader,model):
    if loader.dataset.train:
        print("accuracy for train dataset")
    else:
        print("accuracy for test datsets")
    num_correct = 0
    num_samples=0
    model.eval()
    
    with torch.no_grad():
        for x,y in loader:
            x = x.to(device=device).squeeze(1)
            y = y.to(device = device)
           # x =x.reshape(x.shape[0] ,-1)
            scores = model(x)
            _,predictions = scores.max(1)
            num_correct += (predictions==y).sum()
            num_samples += predictions.size(0)
        print(f'got {num_correct}/{num_samples} with accuraccy  {float(num_correct)/float(num_samples)*100:.2f}'
        )
    model.train()


In [130]:
check_accuracy(test_loader , model)

accuracy for test datsets
got 9880/10000 with accuraccy  98.80
